In [1]:
import os
from fractions import Fraction
import pandas as pd

In [2]:
HYPHEN_LINE = '-' * 50

# calculate Likelihood probability


In [3]:

def calc_likelihood_probs(data):
    likelihoods = {}  
    yes_group = data[data['Play Tennis'] == 'Yes']
    no_group = data[data['Play Tennis'] == 'No']
    classes = {'Yes': yes_group, 'No': no_group}  
    for class_label, group in classes.items():
        feature_likelihoods = {}  
        for feature in group.columns[:-1]:
            value_likelihoods = {}  
            for value in group[feature]:
                value_count = len(group[group[feature] == value])
                value_likelihoods[value] = Fraction(value_count, len(group)).limit_denominator()  
            feature_likelihoods[feature] = value_likelihoods
        likelihoods[class_label] = feature_likelihoods
    return likelihoods

# calculate prior probability

In [4]:

def calc_prior_probs(data):
    total_count = len(data)  
    class_labels = data['Play Tennis'].unique()  
    prior_probs = {} 
    for class_label in class_labels:
        class_group = data[data['Play Tennis'] == class_label] 
        class_count = len(class_group)  
        prior_probs[class_label] = class_count / total_count 
    return prior_probs


# pridect function

In [5]:
def predict(test_sample, likelihood_probs, prior_probs):
    yes_prob = prior_probs['Yes']
    no_prob = prior_probs['No']
    for feature, value in test_sample.items():
        if value in likelihood_probs['Yes'][feature]: 
            yes_prob *= likelihood_probs['Yes'][feature][value]  
        else:
            yes_prob *= 0  
        if value in likelihood_probs['No'][feature]:  
            no_prob *= likelihood_probs['No'][feature][value]  
        else:
            no_prob *= 0  

        print(f"{HYPHEN_LINE}\nP({feature}={value}|Play=Yes) = {likelihood_probs['Yes'][feature][value]}")
        print(f"P({feature}={value}|Play=No) = {likelihood_probs['No'][feature][value]}")

    print(
        f"{HYPHEN_LINE}\nP(Play=Yes) = {Fraction(prior_probs['Yes']).limit_denominator()} \nP(Play=No) = {Fraction(prior_probs['No']).limit_denominator()}")

    print(f"{HYPHEN_LINE}\nP(Yes|X`) = {round(yes_prob, 4)} \nP(No|X`) = {round(no_prob, 4)} \n{HYPHEN_LINE}")
    return 'Yes' if yes_prob > no_prob else 'No' 

In [10]:
if __name__ == "__main__":
    data = pd.read_csv(r'PlayTennis.csv')
    
    likelihood_probs = calc_likelihood_probs(data)
    print(HYPHEN_LINE)
    
    for feature, values in likelihood_probs['Yes'].items():
        df = pd.DataFrame({'Yes': values, 'No': likelihood_probs['No'][feature]}, columns=likelihood_probs.keys(),
                          index=values.keys())
        df.columns.name = feature
        df = df.fillna(0)
        df = df.sort_index(axis=0)
        print(f"{df}\n{HYPHEN_LINE}")
    
    prior_probs = calc_prior_probs(data)
    test_sample = {'Outlook': 'Sunny', 'Temperature': 'Cool', 'Humidity': 'High', 'Wind': 'Strong'}  
    
    print("\nX`=", test_sample, "\n")  
   
    predicted_class = predict(test_sample, likelihood_probs, prior_probs )
    print("Predicted class of X`:", predicted_class)

--------------------------------------------------
Outlook   Yes   No
Overcast  4/9    0
Rain      1/3  2/5
Sunny     2/9  3/5
--------------------------------------------------
Temperature  Yes   No
Cool         1/3  1/5
Hot          2/9  2/5
Mild         4/9  2/5
--------------------------------------------------
Humidity  Yes   No
High      1/3  4/5
Normal    2/3  1/5
--------------------------------------------------
Wind    Yes   No
Strong  1/3  3/5
Weak    2/3  2/5
--------------------------------------------------

X`= {'Outlook': 'Sunny', 'Temperature': 'Cool', 'Humidity': 'High', 'Wind': 'Strong'} 

--------------------------------------------------
P(Outlook=Sunny|Play=Yes) = 2/9
P(Outlook=Sunny|Play=No) = 3/5
--------------------------------------------------
P(Temperature=Cool|Play=Yes) = 1/3
P(Temperature=Cool|Play=No) = 1/5
--------------------------------------------------
P(Humidity=High|Play=Yes) = 1/3
P(Humidity=High|Play=No) = 4/5
------------------------------------